In [1]:
import os
import json

# --- Dosya yolları ---
annotations_path = 'coco/annotations/mini_instances_val2017.json'  # JSON dosyanın yolu
images_folder = 'val2017'  # Resimlerin olduğu klasör
labels_folder = 'labels'  # Yeni label dosyalarının yazılacağı klasör

# --- labels klasörünü oluştur ---
os.makedirs(labels_folder, exist_ok=True)

# --- COCO JSON dosyasını oku ---
with open(annotations_path, 'r') as f:
    coco_data = json.load(f)

# --- image_id -> file_name eşlemesi yap ---
id2filename = {img['id']: img['file_name'].split('.')[0] for img in coco_data['images']}

# --- Kategorileri oku ---
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

# --- Tüm annotation'ları işle ve YOLO formatına çevir ---
annotations_per_image = {}

for ann in coco_data['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    bbox = ann['bbox']  # COCO formatı: [x_min, y_min, width, height]

    # Görüntü boyutlarını bul
    img_info = next((img for img in coco_data['images'] if img['id'] == image_id), None)
    if img_info is None:
        continue
    img_width = img_info['width']
    img_height = img_info['height']

    # YOLO formatına çevir (normalize edilmiş)
    x_center = (bbox[0] + bbox[2] / 2) / img_width
    y_center = (bbox[1] + bbox[3] / 2) / img_height
    width = bbox[2] / img_width
    height = bbox[3] / img_height

    # YOLO satırı oluştur
    yolo_line = f"{category_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    filename = id2filename[image_id]
    if filename not in annotations_per_image:
        annotations_per_image[filename] = []
    annotations_per_image[filename].append(yolo_line)

# --- YOLO txt dosyalarını yaz ---
for filename, annotations in annotations_per_image.items():
    txt_path = os.path.join(labels_folder, f"{filename}.txt")
    with open(txt_path, 'w') as f:
        f.write('\n'.join(annotations))

print("✅ Tüm YOLO formatlı .txt dosyaları oluşturuldu.")

# --- İstersen data.yaml dosyasını da yazabiliriz ---
yaml_content = f"""\
path: .
train: images/val
val: images/val

names:
"""

for cat_id, name in categories.items():
    yaml_content += f"  {cat_id}: {name}\n"

# data.yaml dosyasını kaydet
yaml_save_path = 'data.yaml'
with open(yaml_save_path, 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml dosyası oluşturuldu.")

✅ Tüm YOLO formatlı .txt dosyaları oluşturuldu.
✅ data.yaml dosyası oluşturuldu.
